In [1]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, TrainerCallback

In [2]:
# datasets
ds = load_dataset("yahma/alpaca-cleaned", split="train[:8000]")
ds

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 8000
})

In [3]:
# show the data
ds[:3]

{'input': ['', '', ''],
 'instruction': ['Give three tips for staying healthy.',
  'What are the three primary colors?',
  'Describe the structure of an atom.'],
 'output': ['1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
  'The three primary colors are red, blue, and yellow. These col

In [4]:
# download tokenizer
tokenizer = AutoTokenizer.from_pretrained("../bloom-1b1")
tokenizer

BloomTokenizerFast(name_or_path='../bloom-1b1', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
# datasets process function
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
# apply the process function to datasets
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8000
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: What are the three primary colors?\n\nAssistant: The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).</s>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).</s>'

In [9]:
# download model
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b1", low_cpu_mem_usage=True)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

In [10]:
# set prefix tuning config
from peft import PrefixTuningConfig, get_peft_model, TaskType

config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10, prefix_projection=True)
config

PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_hidden_size=None, prefix_projection=True)

In [11]:
model = get_peft_model(model, config)

In [12]:
model.prompt_encoder

ModuleDict(
  (default): PrefixEncoder(
    (embedding): Embedding(10, 1536)
    (transform): Sequential(
      (0): Linear(in_features=1536, out_features=1536, bias=True)
      (1): Tanh()
      (2): Linear(in_features=1536, out_features=73728, bias=True)
    )
  )
)

In [13]:
model.print_trainable_parameters()

trainable params: 115,696,128 || all params: 1,181,010,432 || trainable%: 9.796367996857796


In [14]:
# set training arguments
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

In [15]:
# create a trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

C:\Users\10021\anaconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
# store the losses
class LossLoggingCallback(TrainerCallback):
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'loss' in logs:
            self.losses.append(logs['loss'])
            with open(f"{self.output_dir}/losses.txt", "a") as f:
                f.write(f"{state.global_step}: {logs['loss']}\n")

trainer.add_callback(LossLoggingCallback(output_dir="./"))
# start training
trainer.train()

Step,Training Loss
10,3.117900
20,2.442200
30,2.243400
40,2.267900
50,2.235800
60,1.950200
70,1.962700
80,2.036000
90,2.037700
100,1.934900


TrainOutput(global_step=1000, training_loss=1.835702431678772, metrics={'train_runtime': 11485.0448, 'train_samples_per_second': 0.697, 'train_steps_per_second': 0.087, 'total_flos': 4491070176079872.0, 'train_loss': 1.835702431678772, 'epoch': 1.0})

In [17]:
# model inference
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("How to prepare an exam?", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: How to prepare an exam?\n\nAssistant: Getting a test, preparing for it, and making plans for it are only two of the steps to take when preparing for an exam. These steps will help you achieve better results when participating in the exam. Here are some important strategies for preparing for your exam.'